In [3]:
import pandas as pd
import re

def analyze_noise(df, df_name):
    """실제 데이터에 어떤 노이즈가 있는지 확인"""
    print(f"\n{'='*80}")
    print(f"{df_name} 분석 ({len(df)} samples)")
    print(f"{'='*80}")
    
    dialogues = df['dialogue'].astype(str)
    
    # 1. \\n 체크
    backslash_n_count = dialogues.str.contains(r'\\n', regex=True).sum()
    print(f"1. \\n 포함: {backslash_n_count}개 ({backslash_n_count/len(df)*100:.1f}%)")
    if backslash_n_count > 0:
        sample_idx = dialogues[dialogues.str.contains(r'\\n', regex=True)].index[0]
        print(f"   샘플: {dialogues.iloc[sample_idx][:100]}...")
    
    # 2. <br> 태그 체크
    br_count = dialogues.str.contains(r'<br', regex=True, flags=re.IGNORECASE).sum()
    print(f"2. <br> 태그 포함: {br_count}개 ({br_count/len(df)*100:.1f}%)")
    if br_count > 0:
        sample_idx = dialogues[dialogues.str.contains(r'<br', regex=True, flags=re.IGNORECASE)].index[0]
        print(f"   샘플: {dialogues.iloc[sample_idx][:100]}...")
    
    # 3. 연속 공백 체크 (3개 이상)
    multi_space_count = dialogues.str.contains(r'   +', regex=True).sum()
    print(f"3. 연속 공백(3개+): {multi_space_count}개 ({multi_space_count/len(df)*100:.1f}%)")
    if multi_space_count > 0:
        sample_idx = dialogues[dialogues.str.contains(r'   +', regex=True)].index[0]
        print(f"   샘플: {dialogues.iloc[sample_idx][:100]}...")
    
    # 4. 연속 개행 체크 (3개 이상)
    multi_newline_count = dialogues.str.contains(r'\n\n\n+', regex=True).sum()
    print(f"4. 연속 개행(3개+): {multi_newline_count}개 ({multi_newline_count/len(df)*100:.1f}%)")
    if multi_newline_count > 0:
        sample_idx = dialogues[dialogues.str.contains(r'\n\n\n+', regex=True)].index[0]
        print(f"   샘플: {dialogues.iloc[sample_idx][:100]}...")
    
    # 5. Person 태그 형태 분석
    print(f"\n5. Person 태그 형태:")
    
    # 정규 형태: #Person1#
    regular_pattern = dialogues.str.contains(r'#Person\d+#', regex=True).sum()
    print(f"   - #Person1# 형태: {regular_pattern}개")
    
    # 공백 있는 형태: # Person 1 #
    space_pattern = dialogues.str.contains(r'#\s+Person\s+\d+\s+#', regex=True).sum()
    print(f"   - # Person 1 # 형태: {space_pattern}개")
    
    # 소문자 형태: #person1#
    lower_pattern = dialogues.str.contains(r'#person\d+#', regex=True).sum()
    print(f"   - #person1# 형태: {lower_pattern}개")
    
    # 불규칙 형태 (전체)
    irregular_pattern = dialogues.str.contains(r'#\s*[Pp]erson\s*\d+\s*#', regex=True).sum()
    print(f"   - 총 Person 태그 포함: {irregular_pattern}개 ({irregular_pattern/len(df)*100:.1f}%)")
    
    # 6. 앞뒤 공백 체크
    leading_space = dialogues.str.match(r'^\s+').sum()
    trailing_space = dialogues.str.match(r'.*\s+$').sum()
    print(f"\n6. 앞쪽 공백 포함: {leading_space}개 ({leading_space/len(df)*100:.1f}%)")
    print(f"   뒤쪽 공백 포함: {trailing_space}개 ({trailing_space/len(df)*100:.1f}%)")
    
    # 7. 특수문자 체크
    special_chars = dialogues.str.contains(r'[^\w\s가-힣#:\n\-.,!?()\'\"@]', regex=True).sum()
    print(f"\n7. 특수문자 포함: {special_chars}개 ({special_chars/len(df)*100:.1f}%)")
    
    # 8. 길이 통계
    print(f"\n8. 대화 길이 통계:")
    lengths = dialogues.str.len()
    print(f"   - 최소: {lengths.min()}자")
    print(f"   - 최대: {lengths.max()}자")
    print(f"   - 평균: {lengths.mean():.1f}자")
    print(f"   - 중앙값: {lengths.median():.1f}자")
    
    # 9. 샘플 출력
    print(f"\n9. 실제 샘플 (첫 3개):")
    for i in range(min(3, len(df))):
        print(f"\n   [{i+1}] fname: {df.iloc[i]['fname']}")
        print(f"       dialogue (첫 150자): {repr(dialogues.iloc[i][:150])}...")
        if 'summary' in df.columns:
            print(f"       summary: {df.iloc[i]['summary']}")
    
    return {
        'total': len(df),
        'backslash_n': backslash_n_count,
        'br_tag': br_count,
        'multi_space': multi_space_count,
        'multi_newline': multi_newline_count,
        'person_tag': irregular_pattern,
        'leading_space': leading_space,
        'trailing_space': trailing_space,
        'special_chars': special_chars
    }


# ============================================================
# 실행 코드
# ============================================================

print("="*80)
print("전체 데이터셋 노이즈 분석")
print("="*80)

# 데이터 로드
train_df = pd.read_csv('/home/NLP_contest/NPL_contest/data/train.csv')
dev_df = pd.read_csv('/home/NLP_contest/NPL_contest/data/dev.csv')
test_df = pd.read_csv('/home/NLP_contest/NPL_contest/data/test.csv')

# 각 데이터셋 분석
train_stats = analyze_noise(train_df, "Train")
dev_stats = analyze_noise(dev_df, "Dev")
test_stats = analyze_noise(test_df, "Test")

# 증강 데이터도 확인 (있으면)
try:
    augmented_df = pd.read_csv('/home/NLP_contest/NPL_contest/augmented_data/high_quality_augmented.csv')
    aug_stats = analyze_noise(augmented_df, "Augmented (고품질)")
except FileNotFoundError:
    print("\n⚠️ 증강 데이터 파일이 없습니다.")
    aug_stats = None

# ============================================================
# 종합 요약
# ============================================================

print("\n" + "="*80)
print("종합 요약")
print("="*80)

summary_table = pd.DataFrame({
    'Train': train_stats,
    'Dev': dev_stats,
    'Test': test_stats
})

if aug_stats:
    summary_table['Augmented'] = aug_stats

print(summary_table.T)

# 전처리 필요 여부 판단
print("\n" + "="*80)
print("전처리 필요 여부 판단")
print("="*80)

total_samples = train_stats['total'] + dev_stats['total'] + test_stats['total']
total_backslash_n = train_stats['backslash_n'] + dev_stats['backslash_n'] + test_stats['backslash_n']
total_br = train_stats['br_tag'] + dev_stats['br_tag'] + test_stats['br_tag']
total_multi_space = train_stats['multi_space'] + dev_stats['multi_space'] + test_stats['multi_space']

print(f"\n전체 샘플: {total_samples}개")
print(f"\n노이즈 비율:")
print(f"  - \\n: {total_backslash_n}개 ({total_backslash_n/total_samples*100:.2f}%)")
print(f"  - <br>: {total_br}개 ({total_br/total_samples*100:.2f}%)")
print(f"  - 연속 공백: {total_multi_space}개 ({total_multi_space/total_samples*100:.2f}%)")

print("\n" + "="*80)
print("권장 사항:")
print("="*80)

# 판단 기준: 1% 이상이면 전처리 권장
needs_preprocessing = []

if total_backslash_n / total_samples > 0.01:
    needs_preprocessing.append("\\n → \\n 변환")
    print("✅ \\n 전처리 필요 (1% 이상)")
else:
    print("❌ \\n 전처리 불필요 (1% 미만)")

if total_br / total_samples > 0.01:
    needs_preprocessing.append("<br> → \\n 변환")
    print("✅ <br> 전처리 필요 (1% 이상)")
else:
    print("❌ <br> 전처리 불필요 (1% 미만)")

if total_multi_space / total_samples > 0.01:
    needs_preprocessing.append("연속 공백 제거")
    print("✅ 연속 공백 전처리 필요 (1% 이상)")
else:
    print("❌ 연속 공백 전처리 불필요 (1% 미만)")

print("\n" + "="*80)
if needs_preprocessing:
    print("🔧 전처리 필요 항목:")
    for item in needs_preprocessing:
        print(f"   - {item}")
else:
    print("✨ 데이터가 매우 깨끗합니다! 전처리 불필요")
    print("   → 바로 모델 학습 진행 가능")
print("="*80)

전체 데이터셋 노이즈 분석

Train 분석 (12457 samples)
1. \n 포함: 1개 (0.0%)
   샘플: #Person1#: 저, 불만이 있어요. 열 분 동안 테이블에서 기다렸는데, 웨이터가 드디어 와서 주문을 받았어요. 그런데 나온 음식이 제가 주문한 게 아니더라고요.\n#Perso...
2. <br> 태그 포함: 1개 (0.0%)
   샘플: #Person1#: 요즘 잘 지내고 있어요?<br>#Person2#: 제 코치가 제 혈압을 체크해 달라고 부탁했어요.<br>#Person1#: 전에 고혈압 있다고 들은 적 있나요?...
3. 연속 공백(3개+): 0개 (0.0%)
4. 연속 개행(3개+): 0개 (0.0%)

5. Person 태그 형태:
   - #Person1# 형태: 12457개
   - # Person 1 # 형태: 0개
   - #person1# 형태: 0개
   - 총 Person 태그 포함: 12457개 (100.0%)

6. 앞쪽 공백 포함: 0개 (0.0%)
   뒤쪽 공백 포함: 0개 (0.0%)

7. 특수문자 포함: 642개 (5.2%)

8. 대화 길이 통계:
   - 최소: 84자
   - 최대: 2165자
   - 평균: 406.1자
   - 중앙값: 369.0자

9. 실제 샘플 (첫 3개):

   [1] fname: train_0
       dialogue (첫 150자): '#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? \n#Person2#: 건강검진을 받으려고 왔어요. \n#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. \n#Person'...
       summary: Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필요성을 안내받고 흡연 습관 개선을 위한 도움을 제안받았습니다.

   [2] fname: train_1
       dialogue (첫 15

In [4]:
import pandas as pd

# 데이터 로드
train = pd.read_csv('/home/NLP_contest/NPL_contest/data/train.csv')
dev = pd.read_csv('/home/NLP_contest/NPL_contest/data/dev.csv')
test = pd.read_csv('/home/NLP_contest/NPL_contest/data/test.csv')

print("="*80)
print("데이터셋 노이즈 요약")
print("="*80)

datasets = {
    'Train': train,
    'Dev': dev,
    'Test': test
}

summary = []

for name, df in datasets.items():
    backslash_n = df['dialogue'].str.contains(r'\\n', regex=True).sum()
    br_tag = df['dialogue'].str.contains(r'<br', regex=True, case=False).sum()
    multi_space = df['dialogue'].str.contains(r'   +', regex=True).sum()
    person_tag = df['dialogue'].str.contains(r'#Person\d+#', regex=True).sum()
    
    summary.append({
        'Dataset': name,
        'Total': len(df),
        '\\n': backslash_n,
        '<br>': br_tag,
        '연속공백': multi_space,
        'Person태그': person_tag
    })
    
    print(f"\n{name}:")
    print(f"  총 샘플: {len(df)}")
    print(f"  \\n: {backslash_n}개 ({backslash_n/len(df)*100:.2f}%)")
    print(f"  <br>: {br_tag}개 ({br_tag/len(df)*100:.2f}%)")
    print(f"  연속 공백: {multi_space}개 ({multi_space/len(df)*100:.2f}%)")
    print(f"  Person 태그: {person_tag}개 ({person_tag/len(df)*100:.2f}%)")

# 표로 정리
summary_df = pd.DataFrame(summary)
print("\n" + "="*80)
print("종합 표")
print("="*80)
print(summary_df.to_string(index=False))

# 전체 통계
total_samples = sum([len(df) for df in datasets.values()])
total_backslash_n = sum([df['dialogue'].str.contains(r'\\n', regex=True).sum() for df in datasets.values()])
total_br = sum([df['dialogue'].str.contains(r'<br', regex=True, case=False).sum() for df in datasets.values()])

print("\n" + "="*80)
print("전체 통계")
print("="*80)
print(f"전체 샘플: {total_samples}개")
print(f"\\n 총: {total_backslash_n}개 ({total_backslash_n/total_samples*100:.3f}%)")
print(f"<br> 총: {total_br}개 ({total_br/total_samples*100:.3f}%)")

print("\n" + "="*80)
print("결론")
print("="*80)
if total_backslash_n / total_samples < 0.01 and total_br / total_samples < 0.01:
    print("✨ 모든 데이터셋이 매우 깨끗합니다!")
    print("   전처리 불필요 → 바로 학습 가능")
else:
    print("⚠️ 일부 전처리 필요")
print("="*80)

데이터셋 노이즈 요약

Train:
  총 샘플: 12457
  \n: 1개 (0.01%)
  <br>: 1개 (0.01%)
  연속 공백: 0개 (0.00%)
  Person 태그: 12457개 (100.00%)

Dev:
  총 샘플: 499
  \n: 0개 (0.00%)
  <br>: 0개 (0.00%)
  연속 공백: 0개 (0.00%)
  Person 태그: 499개 (100.00%)

Test:
  총 샘플: 499
  \n: 0개 (0.00%)
  <br>: 0개 (0.00%)
  연속 공백: 0개 (0.00%)
  Person 태그: 499개 (100.00%)

종합 표
Dataset  Total  \n  <br>  연속공백  Person태그
  Train  12457   1     1     0     12457
    Dev    499   0     0     0       499
   Test    499   0     0     0       499

전체 통계
전체 샘플: 13455개
\n 총: 1개 (0.007%)
<br> 총: 1개 (0.007%)

결론
✨ 모든 데이터셋이 매우 깨끗합니다!
   전처리 불필요 → 바로 학습 가능


In [5]:
"""
KoT5 학습 - 고품질 증강 데이터 활용
필터링된 증강 데이터와 원본 데이터를 결합하여 학습
"""

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    AdamW,
    get_cosine_schedule_with_warmup
)
from tqdm import tqdm
import os
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


# ========================================
# Config 설정
# ========================================
class Config:
    # 경로
    train_path = '/home/NLP_contest/NPL_contest/data/train.csv'  # 원본 train 데이터
    augmented_path = '/home/NLP_contest/NPL_contest/augmented_data/high_quality_augmented.csv'  # 필터링된 증강 데이터
    dev_path = '/home/NLP_contest/NPL_contest/data/dev.csv'  # dev 경로 확인 필요
    test_path = '/home/NLP_contest/NPL_contest/data/test.csv'  # test 경로 확인 필요
    output_dir = '/home/NLP_contest/NPL_contest/outputs'
    model_save_path = '/home/NLP_contest/NPL_contest/models/kot5_with_filtered_augmentation'
    
    # 모델 설정
    model_name = 'KETI-AIR/ke-t5-base'  # KoT5 base 모델
    max_input_length = 512  # 입력 최대 길이
    max_target_length = 128  # 출력 최대 길이
    
    # 학습 설정
    batch_size = 8  # 배치 크기 (GPU 메모리에 따라 조정)
    num_epochs = 6  # 학습 에폭 수
    learning_rate = 1e-4  # 학습률
    warmup_ratio = 0.1  # Warmup 비율
    weight_decay = 0.01  # Weight decay
    max_grad_norm = 1.0  # Gradient clipping
    label_smoothing = 0.1  # Label smoothing
    
    # Early stopping
    patience = 3  # Early stopping patience
    
    # 생성 설정
    num_beams = 4  # Beam search 크기
    length_penalty = 1.0  # 길이 페널티
    no_repeat_ngram_size = 4  # N-gram 반복 방지
    min_length = 10  # 최소 생성 길이
    max_gen_length = 64  # 최대 생성 길이
    
    # 증강 데이터 사용 설정
    use_augmentation = True  # 증강 데이터 사용 여부
    augmentation_ratio = 1.0  # 증강 데이터 사용 비율 (1.0 = 전체 사용)

config = Config()

# 디렉토리 생성
os.makedirs(config.output_dir, exist_ok=True)  # 출력 디렉토리 생성
os.makedirs(config.model_save_path, exist_ok=True)  # 모델 저장 디렉토리 생성


# ========================================
# 데이터 로드 및 결합
# ========================================
print("\n" + "="*80)
print("데이터 로드 및 결합")
print("="*80)

# 원본 train 데이터 로드
train_df = pd.read_csv(config.train_path)
print(f"원본 Train 데이터: {len(train_df)} samples")

# 증강 데이터 로드 및 결합
if config.use_augmentation:
    try:
        # 필터링된 고품질 증강 데이터 로드
        augmented_df = pd.read_csv(config.augmented_path)
        print(f"고품질 증강 데이터: {len(augmented_df)} samples")
        
        # 증강 데이터 비율 적용 (필요시)
        if config.augmentation_ratio < 1.0:
            sample_size = int(len(augmented_df) * config.augmentation_ratio)
            augmented_df = augmented_df.sample(n=sample_size, random_state=42)
            print(f"증강 데이터 샘플링: {len(augmented_df)} samples ({config.augmentation_ratio*100:.0f}%)")
        
        # 증강 데이터 표시 컬럼 추가 (디버깅용)
        train_df['is_augmented'] = False
        augmented_df['is_augmented'] = True
        
        # 원본 + 증강 데이터 결합
        train_df = pd.concat([train_df, augmented_df], ignore_index=True)
        
        # 데이터 셔플
        train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        print(f"최종 Train 데이터: {len(train_df)} samples")
        print(f"  - 원본: {(~train_df['is_augmented']).sum()} samples")
        print(f"  - 증강: {train_df['is_augmented'].sum()} samples")
        
    except FileNotFoundError:
        print(f"⚠️ 증강 데이터 파일을 찾을 수 없습니다: {config.augmented_path}")
        print("원본 데이터만 사용합니다.")
        train_df['is_augmented'] = False
else:
    print("증강 데이터를 사용하지 않습니다.")
    train_df['is_augmented'] = False

# Dev, Test 데이터 로드
dev_df = pd.read_csv(config.dev_path)
test_df = pd.read_csv(config.test_path)

print(f"\nDev: {len(dev_df)} samples")
print(f"Test: {len(test_df)} samples")
print("="*80)


# ========================================
# 데이터셋 클래스
# ========================================
class DialogueSummarizationDataset(Dataset):
    """대화 요약 데이터셋"""
    
    def __init__(self, df, tokenizer, max_input_length, max_target_length, is_test=False):
        """
        Args:
            df: 데이터프레임
            tokenizer: T5 토크나이저
            max_input_length: 입력 최대 길이
            max_target_length: 출력 최대 길이
            is_test: 테스트 모드 여부
        """
        self.df = df
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # 대화 텍스트 가져오기
        dialogue = str(row['dialogue'])
        
        # T5는 task prefix를 사용 (예: "summarize: ")
        input_text = f"summarize: {dialogue}"
        
        # 입력 토큰화
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_input_length,
            padding='max_length',  # 최대 길이로 패딩
            truncation=True,  # 최대 길이 초과시 자르기
            return_tensors='pt'
        )
        
        input_ids = input_encoding['input_ids'].squeeze()  # (seq_len,)
        attention_mask = input_encoding['attention_mask'].squeeze()  # (seq_len,)
        
        # 테스트 모드면 fname도 반환
        if self.is_test:
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'fname': row['fname']
            }
        
        # 요약문 토큰화 (학습용)
        summary = str(row['summary'])
        
        target_encoding = self.tokenizer(
            summary,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        labels = target_encoding['input_ids'].squeeze()  # (seq_len,)
        
        # 패딩 토큰은 -100으로 변경 (loss 계산에서 제외)
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }


# ========================================
# 모델 및 토크나이저 로드
# ========================================
print("\n" + "="*80)
print("KoT5 모델 및 토크나이저 로드")
print("="*80)

# 토크나이저 로드
tokenizer = T5TokenizerFast.from_pretrained(config.model_name)
print(f"Tokenizer loaded: {config.model_name}")
print(f"Vocab size: {len(tokenizer)}")

# Person 태그 추가
special_tokens = [f'#Person{i}#' for i in range(1, 8)]
num_added = tokenizer.add_tokens(special_tokens)
print(f"Added {num_added} special tokens: {special_tokens}")

# 모델 로드
model = T5ForConditionalGeneration.from_pretrained(config.model_name)
model.resize_token_embeddings(len(tokenizer))  # 토크나이저 크기에 맞게 임베딩 조정
model = model.to(device)

# 모델 파라미터 정보
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


# ========================================
# 데이터셋 및 데이터로더 생성
# ========================================
print("\n" + "="*80)
print("데이터셋 생성")
print("="*80)

# 데이터셋 생성
train_dataset = DialogueSummarizationDataset(
    train_df, tokenizer, 
    config.max_input_length, 
    config.max_target_length
)

dev_dataset = DialogueSummarizationDataset(
    dev_df, tokenizer,
    config.max_input_length,
    config.max_target_length
)

test_dataset = DialogueSummarizationDataset(
    test_df, tokenizer,
    config.max_input_length,
    config.max_target_length,
    is_test=True
)

# 데이터로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,  # 학습 데이터는 셔플
    num_workers=2  # 병렬 데이터 로딩
)

dev_loader = DataLoader(
    dev_dataset,
    batch_size=config.batch_size,
    shuffle=False,  # Dev 데이터는 셔플 안 함
    num_workers=2
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,  # Test 데이터는 셔플 안 함
    num_workers=2
)

print(f"Train batches: {len(train_loader)}")
print(f"Dev batches: {len(dev_loader)}")
print(f"Test batches: {len(test_loader)}")


# ========================================
# Optimizer 및 Scheduler
# ========================================
print("\n" + "="*80)
print("학습 설정")
print("="*80)

# Weight decay를 적용하지 않을 파라미터 지정
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters() 
                  if not any(nd in n for nd in no_decay)],
        'weight_decay': config.weight_decay
    },
    {
        'params': [p for n, p in model.named_parameters() 
                  if any(nd in n for nd in no_decay)],
        'weight_decay': 0.0
    }
]

# Optimizer 생성
optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=config.learning_rate,
    eps=1e-8
)

# Scheduler 생성 (Cosine annealing with warmup)
total_steps = len(train_loader) * config.num_epochs
warmup_steps = int(total_steps * config.warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {warmup_steps}")
print(f"Learning rate: {config.learning_rate}")


# ========================================
# Early Stopping 클래스
# ========================================
class EarlyStopping:
    """Early stopping을 위한 클래스"""
    
    def __init__(self, patience=3, min_delta=0.0, mode='max'):
        """
        Args:
            patience: 개선이 없어도 기다릴 에폭 수
            min_delta: 개선으로 인정할 최소 변화량
            mode: 'max' (높을수록 좋음) or 'min' (낮을수록 좋음)
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0  # 개선 없는 에폭 카운터
        self.best_score = None
        self.early_stop = False
    
    def __call__(self, score):
        """
        현재 점수를 체크하고 개선 여부 반환
        
        Args:
            score: 현재 에폭의 점수
            
        Returns:
            bool: 개선되었으면 True
        """
        if self.best_score is None:
            self.best_score = score
            return True
        
        # 개선 여부 체크
        if self.mode == 'max':
            improved = score > self.best_score + self.min_delta
        else:
            improved = score < self.best_score - self.min_delta
        
        if improved:
            self.best_score = score
            self.counter = 0
            return True
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
            return False


# ========================================
# 학습 함수
# ========================================
def train_epoch(model, loader, optimizer, scheduler, device, config):
    """
    한 에폭 학습
    
    Args:
        model: 학습할 모델
        loader: 학습 데이터로더
        optimizer: Optimizer
        scheduler: Learning rate scheduler
        device: 디바이스 (cuda/cpu)
        config: 설정
        
    Returns:
        float: 평균 loss
    """
    model.train()  # 학습 모드
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(loader, desc="Training")
    
    for step, batch in enumerate(progress_bar):
        # 데이터를 디바이스로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        total_loss += loss.item()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
        
        # Optimizer step
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        # 진행상황 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
    
    return total_loss / len(loader)


# ========================================
# ROUGE 평가 함수
# ========================================
def evaluate_model_on_dev(model, dev_loader, tokenizer, device, config):
    """
    Dev 데이터로 모델 평가 (ROUGE 계산)
    
    Args:
        model: 평가할 모델
        dev_loader: Dev 데이터로더
        tokenizer: 토크나이저
        device: 디바이스
        config: 설정
        
    Returns:
        dict: ROUGE 점수들
    """
    from rouge import Rouge
    
    # 한국어 형태소 분석기 (kiwipiepy)
    try:
        from kiwipiepy import Kiwi
        kiwi = Kiwi()
        use_morpheme = True
    except:
        use_morpheme = False
    
    def tokenize_for_rouge(text):
        """ROUGE 계산을 위한 토큰화"""
        text = str(text).strip()
        if not text:
            return ""
        
        if use_morpheme:
            # 형태소 분석
            tokens = kiwi.tokenize(text)
            morphemes = [token.form for token in tokens]
            return ' '.join(morphemes)
        else:
            # 단순 공백 분리
            return ' '.join(text.split())
    
    print("\nDev 데이터 평가 (ROUGE 계산)")
    print("-" * 80)
    
    model.eval()  # 평가 모드
    predictions = []
    references = []
    
    # 요약문 생성
    with torch.no_grad():
        for batch in tqdm(dev_loader, desc="Generating summaries"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Beam search로 요약문 생성
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=config.max_gen_length,
                min_length=config.min_length,
                num_beams=config.num_beams,
                length_penalty=config.length_penalty,
                no_repeat_ngram_size=config.no_repeat_ngram_size,
                early_stopping=True
            )
            
            # 생성된 텍스트 디코딩
            for gen_ids, label_ids in zip(generated_ids, batch['labels']):
                pred = tokenizer.decode(gen_ids, skip_special_tokens=True)
                label_ids = label_ids[label_ids != -100]
                ref = tokenizer.decode(label_ids, skip_special_tokens=True)
                
                predictions.append(pred)
                references.append(ref)
    
    # ROUGE 계산
    rouge = Rouge()
    
    print("토큰화 및 ROUGE 계산 중...")
    pred_tokenized = [tokenize_for_rouge(p) for p in predictions]
    ref_tokenized = [tokenize_for_rouge(r) for r in references]
    
    # 빈 문자열 제거
    valid_pairs = [(p, r) for p, r in zip(pred_tokenized, ref_tokenized) if p and r]
    
    if not valid_pairs:
        return {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0, 'final_score': 0.0}
    
    pred_valid, ref_valid = zip(*valid_pairs)
    
    try:
        # ROUGE 점수 계산
        scores = rouge.get_scores(list(pred_valid), list(ref_valid))
        
        # F1 점수 평균
        rouge1 = np.mean([s['rouge-1']['f'] for s in scores]) * 100
        rouge2 = np.mean([s['rouge-2']['f'] for s in scores]) * 100
        rougel = np.mean([s['rouge-l']['f'] for s in scores]) * 100
        
        results = {
            'rouge-1': rouge1,
            'rouge-2': rouge2,
            'rouge-l': rougel,
            'final_score': rouge1 + rouge2 + rougel
        }
        
        print(f"\nROUGE-1 F1: {rouge1:.2f}")
        print(f"ROUGE-2 F1: {rouge2:.2f}")
        print(f"ROUGE-L F1: {rougel:.2f}")
        print(f"{'='*80}")
        print(f"Final Score: {results['final_score']:.2f}")
        print(f"{'='*80}")
        
        return results
    
    except Exception as e:
        print(f"ROUGE 계산 중 오류: {e}")
        return {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0, 'final_score': 0.0}


# ========================================
# 학습 루프
# ========================================
print("\n" + "="*80)
print("KoT5 학습 시작 (고품질 증강 데이터 활용)")
print("="*80)

best_rouge_score = 0.0  # 최고 ROUGE 점수 저장
early_stopping = EarlyStopping(patience=config.patience, mode='max')

for epoch in range(config.num_epochs):
    print(f"\nEpoch {epoch + 1}/{config.num_epochs}")
    print("-" * 80)
    
    # 학습
    train_loss = train_epoch(
        model, train_loader, optimizer, scheduler, device, config
    )
    
    print(f"Train Loss: {train_loss:.4f}")
    
    # Dev 평가
    rouge_scores = evaluate_model_on_dev(
        model, dev_loader, tokenizer, device, config
    )
    
    current_score = rouge_scores['final_score']
    
    # Best model 저장
    if current_score > best_rouge_score:
        best_rouge_score = current_score
        print(f"✓ New best model! (ROUGE Final: {current_score:.2f})")
        
        # 모델 저장
        model.save_pretrained(config.model_save_path)
        tokenizer.save_pretrained(config.model_save_path)
        print(f"Model saved to {config.model_save_path}")
    
    # Early stopping 체크
    improved = early_stopping(current_score)
    
    if early_stopping.early_stop:
        print(f"\n⚠ Early stopping triggered at epoch {epoch + 1}")
        print(f"Best ROUGE score: {best_rouge_score:.2f}")
        break
    
    if not improved:
        print(f"⚠ No improvement for {early_stopping.counter} epoch(s)")

print("\n" + "="*80)
print("학습 완료!")
print("="*80)
print(f"Best ROUGE Final: {best_rouge_score:.2f}")


# ========================================
# 후처리 함수
# ========================================
def post_process_summary(summary, max_words=25):
    """
    생성된 요약문 후처리
    
    Args:
        summary: 생성된 요약문
        max_words: 최대 단어 수
        
    Returns:
        str: 후처리된 요약문
    """
    if not summary or not summary.strip():
        return "#Person1#과 #Person2#가 대화합니다."
    
    summary = summary.strip()
    
    # 마침표 추가 (한국어 어미로 끝나면)
    if not summary.endswith('.'):
        if summary.split()[-1][-1] in ['다', '요', '음', '지', '나']:
            summary += '.'
    
    return summary


# ========================================
# 추론 함수
# ========================================
def generate_summaries(model, loader, tokenizer, device, config):
    """
    테스트 데이터에 대한 요약문 생성
    
    Args:
        model: 학습된 모델
        loader: 테스트 데이터로더
        tokenizer: 토크나이저
        device: 디바이스
        config: 설정
        
    Returns:
        tuple: (fnames, summaries)
    """
    model.eval()  # 평가 모드
    
    fnames = []
    summaries = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Generating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Beam search로 요약문 생성
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=config.max_gen_length,
                min_length=config.min_length,
                num_beams=config.num_beams,
                length_penalty=config.length_penalty,
                no_repeat_ngram_size=config.no_repeat_ngram_size,
                early_stopping=True
            )
            
            # 디코딩 및 후처리
            for fname, gen_ids in zip(batch['fname'], generated_ids):
                summary = tokenizer.decode(gen_ids, skip_special_tokens=True)
                summary = post_process_summary(summary, max_words=25)
                
                fnames.append(fname)
                summaries.append(summary)
    
    return fnames, summaries


# ========================================
# 테스트 데이터 추론
# ========================================
print("\n" + "="*80)
print("테스트 데이터 추론")
print("="*80)

# Best model 로드
model = T5ForConditionalGeneration.from_pretrained(config.model_save_path)
model = model.to(device)
tokenizer = T5TokenizerFast.from_pretrained(config.model_save_path)

# 요약문 생성
fnames, summaries = generate_summaries(
    model, test_loader, tokenizer, device, config
)

# 제출 파일 생성
submission_df = pd.DataFrame({
    'fname': fnames,
    'summary': summaries
})

# 제출 파일 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission_filename = f'submission_kot5_filtered_aug_{timestamp}.csv'
submission_path = os.path.join(config.output_dir, submission_filename)
submission_df.to_csv(submission_path, index=False)

print(f"\n제출 파일 저장: {submission_path}")
print(f"생성된 요약 개수: {len(submission_df)}")

# 샘플 출력
print("\n" + "="*80)
print("생성된 요약 샘플")
print("="*80)

for i in range(min(5, len(submission_df))):
    summary = submission_df.iloc[i]['summary']
    word_count = len(summary.split())
    print(f"\n[Sample {i+1}]")
    print(f"Fname: {submission_df.iloc[i]['fname']}")
    print(f"Summary ({word_count} words): {summary}")

# 요약 길이 통계
print("\n" + "="*80)
print("요약 길이 통계")
print("="*80)

summary_lengths = submission_df['summary'].apply(lambda x: len(x.split()))
print(f"평균: {summary_lengths.mean():.1f} 단어")
print(f"중간값: {summary_lengths.median():.1f} 단어")
print(f"최소: {summary_lengths.min()} 단어")
print(f"최대: {summary_lengths.max()} 단어")
print(f"표준편차: {summary_lengths.std():.1f} 단어")

# 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n" + "="*80)
print("KoT5 + 고품질 증강 데이터 파이프라인 완료!")
print("="*80)
print(f"\n저장된 파일:")
print(f"1. 모델: {config.model_save_path}")
print(f"2. 제출 파일: {submission_path}")
print(f"\n기대 성능:")
print(f"- 고품질 증강 데이터로 일반화 성능 향상")
print(f"- 노이즈가 제거되어 더 안정적인 학습")
print(f"- 예상 점수: 50~55점 (필터링된 증강 데이터 효과)")

Using device: cuda
GPU: NVIDIA GeForce RTX 3090
GPU Memory: 23.69 GB

데이터 로드 및 결합
원본 Train 데이터: 12457 samples
고품질 증강 데이터: 3476 samples
최종 Train 데이터: 15933 samples
  - 원본: 12457 samples
  - 증강: 3476 samples

Dev: 499 samples
Test: 499 samples

KoT5 모델 및 토크나이저 로드
Tokenizer loaded: KETI-AIR/ke-t5-base
Vocab size: 64100
Added 7 special tokens: ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#']
Total parameters: 247,463,424
Trainable parameters: 247,463,424

데이터셋 생성
Train batches: 1992
Dev batches: 63
Test batches: 63

학습 설정
Total training steps: 11952
Warmup steps: 1195
Learning rate: 0.0001

KoT5 학습 시작 (고품질 증강 데이터 활용)

Epoch 1/6
--------------------------------------------------------------------------------


Training: 100%|██████████| 1992/1992 [11:51<00:00,  2.80it/s, loss=4.4603, lr=9.87e-05]


Train Loss: 12.3613

Dev 데이터 평가 (ROUGE 계산)
--------------------------------------------------------------------------------


Generating summaries: 100%|██████████| 63/63 [01:03<00:00,  1.01s/it]


토큰화 및 ROUGE 계산 중...

ROUGE-1 F1: 38.78
ROUGE-2 F1: 15.92
ROUGE-L F1: 35.26
Final Score: 89.96
✓ New best model! (ROUGE Final: 89.96)
Model saved to /home/NLP_contest/NPL_contest/models/kot5_with_filtered_augmentation

Epoch 2/6
--------------------------------------------------------------------------------


...

Train Loss: 2.9503

Dev 데이터 평가 (ROUGE 계산)
--------------------------------------------------------------------------------


Generating summaries: 100%|██████████| 63/63 [01:06<00:00,  1.05s/it]


토큰화 및 ROUGE 계산 중...

ROUGE-1 F1: 48.38
ROUGE-2 F1: 23.81
ROUGE-L F1: 42.60
Final Score: 114.79
✓ New best model! (ROUGE Final: 114.79)
Model saved to /home/NLP_contest/NPL_contest/models/kot5_with_filtered_augmentation

학습 완료!
Best ROUGE Final: 114.79

테스트 데이터 추론


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating: 100%|██████████| 63/63 [01:08<00:00,  1.09s/it]



제출 파일 저장: /home/NLP_contest/NPL_contest/outputs/submission_kot5_filtered_aug_20251013_034209.csv
생성된 요약 개수: 499

생성된 요약 샘플

[Sample 1]
Fname: test_0
Summary (29 words): 아이언 Dawson은 #Person1# 에게 사내 메모를 보내달라고 요청합니다. Ms. Dawson은 #Person2# 에게 메시지 프로그램 사용은 제한되지만, #Person1# 은 메시지 프로그램 사용이 제한된다고 설명합니다. #Person2# 는 #Person1# 의 제안에 동의합니다.

[Sample 2]
Fname: test_1
Summary (17 words): 재상은 #Person1# 에게 지하철으로 출퇴근할 것을 제안하지만, #Person1# 은 차를 피하고 싶어합니다. #Person2# 는 대중교통으로 출근하기로 결정합니다.

[Sample 3]
Fname: test_2
Summary (17 words): 아이언은 Kate에게 Masha와 Hero가 이혼했다고 말합니다. Kate는 #Person1# 에게 Masha가 양육권을 가지고, 아이들을 양육할 수 있다고 말합니다.

[Sample 4]
Fname: test_3
Summary (12 words): 재상은 #Person1# 의 생일 파티에 초대하고, #Person1# 은 #Person2# 에게 춤추기로 합니다.

[Sample 5]
Fname: test_4
Summary (20 words): 아이언 공원은 #Person1# 에게 올림픽 스타디움에 대해 이야기합니다. #Person1# 은 #Person2# 에게 좌석이 5000석이라고 설명합니다. #Person2# 는 등산 금기라고 설명합니다.

요약 길이 통계
평균: 18.0 단어
중간값: 18.0 단어
최소: 6 단어
최대: 34 단어
표준편차: 5.6 단어

KoT5 + 고품질 증강 데이터 파이프라인 완료!

저장된 파일:
1. 모델: /ho